# Tabular methods for temporal-difference learning  

# Reference

Richard S. Sutton and Andrew G. Barto (2018). *Reinforcement Learning: An Introduction*. A Bradford Book, Cambridge, MA, USA.

### Import necessary packages

In [1]:
import numpy as np
import gym

Note: The performance of the algorithms can be improved by adjusting parameters.

# Section 1 On policy learning

## Sarsa algorithm

<img src="images/tabular-sarsa.png" width="800" height="800" style="float: left"/>

### Sarsa agent

In [2]:
class SarsaAgent:

    def __init__(self, env, num_grids):
        
        self.env = env
        
        # discretize state space putting a grid
        self.grid_size = (env.observation_space.high - env.observation_space.low) / num_grids        

        # numpy array of size (d1,d2,...,dn)
        table_size = tuple(num_grids) + (env.action_space.n,)
        self.q_table = np.zeros(table_size)
        
        # learning parameters
        self.num_episodes = 5000
        self.epsilon = 0.3
        self.alpha = 0.1 # learning rate
        self.gamma = 0.99 # discount rate
        
        self.verbose = 200
        self.to_show = 500
        self.stats = {'rewards': []}        

    def get_discrete_state(self, state):
        discrete_state = (state - self.env.observation_space.low) / self.grid_size
        return tuple(discrete_state.astype(np.int))  # returns tuple of integers

    def act(self):
        state = self.env.reset()
        done = False
        while not done:
            discrete_state = self.get_discrete_state(state)            
            q_values = self.q_table[discrete_state]
            action = self.epsilon_greedy_selection(q_values, epsilon=0)
            state, reward, done, _ = self.env.step(action)
            self.env.render()
        self.env.close()
            
    def epsilon_greedy_selection(self, q_values, epsilon):
        # select action
        if np.random.random() < epsilon:  # Exploration: randomly selected action
            action = np.random.randint(0, len(q_values))

        else:  # Greedy selection: action with max Q(s,a)
            q_max = np.max(q_values)
            q_max_actions = [i for i, q in enumerate(q_values) if q == q_max]
            action = q_max_actions[np.random.randint(0, len(q_max_actions))]
        return action

    def execute(self):

        # for each episode
        for episode in range(self.num_episodes):
            
            # init episode stats
            self.stats['rewards'].append(0)          
            
            # get initial state
            state = self.env.reset()
            
            # discretize current state
            state = self.get_discrete_state(state)

            # select action based on Q(s,a) for the current state
            action = self.epsilon_greedy_selection(self.q_table[state], self.epsilon)

            # for each time step in the episode (or until terminal state)
            while True:

                # carry out action, observe new state and reward
                new_state, reward, done, _ = self.env.step(action)
                
                # discretize new state
                new_state = self.get_discrete_state(new_state)

                # select action for new state
                new_action = self.epsilon_greedy_selection(self.q_table[new_state], self.epsilon)

                # compute Q(s_new,a) in the new state
                if done:
                    q_new = reward
                else:
                    q_new = reward + self.gamma * self.q_table[new_state][new_action]
                
                # index to look up q_table
                idx = state + (action,)
                
                # get current Q(s,a)
                q_old = self.q_table[idx]

                # update
                self.q_table[idx] = q_old + self.alpha * (q_new - q_old)

                # update episode stats
                self.stats['rewards'][-1] += reward

                # decide whether to continue episode
                if done:
                    break
                else:  # update
                    state = new_state
                    action = new_action

            # print stats on screen
            if self.verbose > 0 and episode % self.verbose == 0:
                print("episode: {}\t reward: {:.2f}".format(episode, self.stats['rewards'][-1]))
                
            # show simulated environment with learned parameters
            if self.to_show > 0 and episode % self.to_show == 0:
                self.act()

### Example

In [3]:
env = gym.make("MountainCar-v0")

agent = SarsaAgent(env=env, num_grids=[20, 20])
agent.execute()
agent.act()

episode: 0	 reward: -200.00
episode: 200	 reward: -200.00
episode: 400	 reward: -200.00
episode: 600	 reward: -200.00
episode: 800	 reward: -200.00
episode: 1000	 reward: -200.00
episode: 1200	 reward: -200.00
episode: 1400	 reward: -200.00
episode: 1600	 reward: -200.00
episode: 1800	 reward: -200.00
episode: 2000	 reward: -200.00
episode: 2200	 reward: -200.00
episode: 2400	 reward: -200.00
episode: 2600	 reward: -200.00
episode: 2800	 reward: -200.00
episode: 3000	 reward: -200.00
episode: 3200	 reward: -161.00
episode: 3400	 reward: -200.00
episode: 3600	 reward: -200.00
episode: 3800	 reward: -199.00
episode: 4000	 reward: -200.00
episode: 4200	 reward: -171.00
episode: 4400	 reward: -200.00
episode: 4600	 reward: -200.00
episode: 4800	 reward: -200.00


# Section 2 Off policy learning

## Q-learning algorithm

<img src="images/tabular-q-learning.png" width="800" height="800" style="float: left"/>

## Q-learning agent 

In [4]:
class QLearningAgent:

    def __init__(self, env, num_grids):
        
        self.env = env
        
        # discretize state space putting a grid
        self.grid_size = (env.observation_space.high - env.observation_space.low) / num_grids        

        # numpy array of size (d1,d2,...,dn)
        table_size = tuple(num_grids) + (env.action_space.n,)
        self.q_table = np.zeros(table_size)
        
        # learning parameters
        self.num_episodes = 5000
        self.epsilon = 0.3
        self.alpha = 0.1 # learning rate
        self.gamma = 0.99 # discount rate
        
        self.verbose = 200
        self.to_show = 500
        self.stats = {'rewards': []}    

    def get_discrete_state(self, state):
        discrete_state = (state - self.env.observation_space.low) / self.grid_size
        return tuple(discrete_state.astype(np.int))  # returns tuple of integers

    def act(self):
        state = self.env.reset()
        done = False
        while not done:
            discrete_state = self.get_discrete_state(state)            
            q_values = self.q_table[discrete_state]
            action = self.epsilon_greedy_selection(q_values, epsilon=0)
            state, reward, done, _ = self.env.step(action)
            self.env.render()
        self.env.close()
            
    def epsilon_greedy_selection(self, q_values, epsilon):
        # select action
        if np.random.random() < epsilon:  # Exploration: randomly selected action
            action = np.random.randint(0, len(q_values))

        else:  # Greedy selection: action with max Q(s,a)
            q_max = np.max(q_values)
            q_max_actions = [i for i, q in enumerate(q_values) if q == q_max]
            action = q_max_actions[np.random.randint(0, len(q_max_actions))]
        return action

    def execute(self):

        # for each episode
        for episode in range(self.num_episodes):
            
            # init episode stats
            self.stats['rewards'].append(0)
            
            # get initial state
            state = self.env.reset()
            
            # discretize current state
            state = self.get_discrete_state(state)

            # for each time step in the episode (or until terminal state)
            while True:

                # select action based on Q(s,a) for the current state
                action = self.epsilon_greedy_selection(self.q_table[state], self.epsilon)

                # carry out action, observe new state and reward
                new_state, reward, done, _ = self.env.step(action)
                
                # discretize new state
                new_state = self.get_discrete_state(new_state)

                # compute Q(s_new,a) in the new state
                if done:
                    q_new = reward
                else:
                    q_new = reward + self.gamma * np.max(self.q_table[new_state])
                
                # index to look up q_table
                idx = state + (action,)
                
                # get current Q(s,a)
                q_old = self.q_table[idx]

                # update
                self.q_table[idx] = q_old + self.alpha * (q_new - q_old)

                # update episode stats
                self.stats['rewards'][-1] += reward

                # decide whether to continue episode
                if done:
                    break
                else:  # update state
                    state = new_state

            if self.verbose > 0 and episode % self.verbose == 0:
                print("episode: {}\t reward: {:.2f}".format(episode, self.stats['rewards'][-1]))
                
            # show simulated environment with learned parameters
            if self.to_show > 0 and episode % self.to_show == 0:
                self.act()       

### Example

In [5]:
env = gym.make("MountainCar-v0")

agent = QLearningAgent(env=env, num_grids=[20, 20])
agent.execute()
agent.act()

episode: 0	 reward: -200.00
episode: 200	 reward: -200.00
episode: 400	 reward: -200.00
episode: 600	 reward: -200.00
episode: 800	 reward: -200.00
episode: 1000	 reward: -200.00
episode: 1200	 reward: -200.00
episode: 1400	 reward: -200.00
episode: 1600	 reward: -200.00
episode: 1800	 reward: -200.00
episode: 2000	 reward: -158.00
episode: 2200	 reward: -200.00
episode: 2400	 reward: -200.00
episode: 2600	 reward: -200.00
episode: 2800	 reward: -200.00
episode: 3000	 reward: -174.00
episode: 3200	 reward: -200.00
episode: 3400	 reward: -200.00
episode: 3600	 reward: -165.00
episode: 3800	 reward: -200.00
episode: 4000	 reward: -200.00
episode: 4200	 reward: -200.00
episode: 4400	 reward: -200.00
episode: 4600	 reward: -200.00
episode: 4800	 reward: -200.00
